# Exercício - Análise de Sentimentos

Gustavo Henrique da Silva Costa Araujo


20210155800

Tive que reduzir a quantidade de linhas utilizadas em certa parte, pois meu computador estava faltando memória para alocar na parte dos vetores.

In [14]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import pandas as pd



In [16]:
dados = pd.read_csv('B2W-Reviews01.csv')
tokens_negacao = ['não', 'nem', 'nunca', 'jamais', 'tampouco']

C:\Users\PC\AppData\Local\Temp\ipykernel_1340\3244800104.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dados = pd.read_csv('B2W-Reviews01.csv')


In [17]:
dados['review_text'] = dados['review_text'].astype(str)
textos = dados['review_text']
textos = textos.dropna()
textos = textos.str.lower()
textos = textos.to_numpy()

In [18]:
def adicionar_nao_prefixo(texto):
    palavras = texto.split()
    novo_texto = []
    negacao_ativa = False
    
    for palavra in palavras:
        if any(token in palavra.lower() for token in tokens_negacao):
            negacao_ativa = True
            nova_palavra = "NAO_" + palavra  
        elif any(p in palavra for p in ['.', ',', '?', '!', ';']):
            negacao_ativa = False
            nova_palavra = palavra  
        elif negacao_ativa:
            nova_palavra = "NAO_" + palavra
        else:
            nova_palavra = palavra  
        novo_texto.append(nova_palavra) 
        
    return ' '.join(novo_texto)

In [19]:
dados['review_text_modificado'] = dados['review_text'].apply(adicionar_nao_prefixo)


In [20]:
dados['review_text_modificado']

0         Estou contente com a compra entrega rápida o ú...
1         Por apenas R$1994.20,eu consegui comprar esse ...
2         SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...
3         MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...
4         A entrega foi no prazo, as americanas estão de...
                                ...                        
132368    Vale muito, estou usando no controle do Xbox e...
132369    Prático e barato, super indico o produto para ...
132370    Chegou antes do prazo previsto e corresponde a...
132371    Material fraco, poderia ser melhor. Ficou deve...
132372    Comprei esse produto, quando chegou estava com...
Name: review_text_modificado, Length: 132373, dtype: object

In [25]:
dadoshead = dados['review_text_modificado'].head(200)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(dadoshead)

In [27]:
vectors = tfidf.toarray()
vectors

array([[0.        , 0.        , 0.        , ..., 0.        , 0.24130963,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [31]:
first = vectors[0]
first[first != 0.0]


array([0.16668988, 0.16988865, 0.36021002, 0.20202563, 0.24130963,
       0.24130963, 0.22797158, 0.12121485, 0.13156065, 0.26010856,
       0.21762578, 0.26010856, 0.19037373, 0.26010856, 0.24130963,
       0.08572481, 0.21762578, 0.15823675, 0.17703567, 0.24130963,
       0.24130963])

In [33]:
first

array([0.        , 0.        , 0.        , ..., 0.        , 0.24130963,
       0.        ])

In [40]:
teste = ['produto ruim, entrega demorada, atendimento péssimo, nunca mais quero']

In [41]:
teste_vector = tfidf_vectorizer.transform(teste)

In [42]:
teste_vector = teste_vector.toarray()[0]

In [43]:
teste_vector[teste_vector != 0.0]

array([0.5094902 , 0.29402288, 0.34803856, 0.19158506, 0.48636854,
       0.5094902 ])

In [44]:
from sklearn.metrics.pairwise import cosine_similarity

In [45]:
similarities = []

for vec in vectors:
    similarities.append(cosine_similarity([teste_vector], [vec]))

In [46]:
clean_similarities = []

for x in similarities:
    clean_similarities.append(x[0][0])

In [47]:
clean_similarities[:10]

[0.055105432742186336,
 0.0,
 0.04090053306060328,
 0.0,
 0.03793761835098016,
 0.015431478581390278,
 0.015448106692148963,
 0.02828675360604344,
 0.0,
 0.021685637781589138]

In [48]:
import numpy as np
np.argsort(clean_similarities)

array([164, 141,  76, 142,  73, 146, 149,  68,  67, 150,  79, 153, 155,
       156,  60, 157, 158,  56, 161, 162,  51,  64,  50, 138, 136, 109,
       110, 102, 101, 198, 112,  97, 116,  95, 137,  94,  91, 119, 124,
       125, 127, 129, 133, 135,  83,  93,  49, 108,  25,  28,  27, 175,
       166,  24,  23,  22, 176, 178,  19,  18, 179, 193,  15, 185,  13,
       190,  10,  30,  29,  40,  46,   1, 196,   3, 170,  42,  41, 194,
        31,  39,   8,  37, 173,  34,  33,  32, 171, 174, 145,  72, 106,
       188, 117,   5,   6,  61,  99,  16,  52,  78,  48,  69, 100,   9,
        45, 105,  57, 169, 199,  55,  62,  96,  26, 191, 130, 165, 172,
       111, 134,  86, 126, 114,  77, 184, 107, 168, 131, 122, 139, 103,
         7, 140,  85, 151, 147,  21,  70, 197,  81, 118,  43,  80, 148,
        88, 195,   4,  58,   2, 187,  59, 159, 144, 152, 167,  36,  35,
        38,   0, 177, 180, 113,  14,  47, 183, 186, 123,  11,  12,  71,
        53, 160, 182, 121, 120, 128,  20, 163,  75,  89, 115, 18

In [51]:
clean_similarities[74]

0.14132479850873486

In [52]:
top_10 = np.argsort(clean_similarities)[-10:]

In [53]:
textos[top_10]

array(['ótimo produto, original, recomendo a todos!!! podem comprar, prazo de entrega dentro do esperado, preço muito atrativo, parabéns as americanas, farei mais compras futuras.',
       'muito bom produto excelente,entrega muito rápido recomendadíssimo',
       'a câmera frontal do meu é terrível, tanto faz tá claro ou escuro, a foto fica ruim de qualquer jeito. muito arrependida!',
       'ótima entrega. parabéns às lojas americanas por cumprir a entrega.',
       'adorei agora faço meus pães em casa com muito mais qualidade e muito mais saudáveis',
       'loja ruim cancelou a compra sem solicitaçao de cancelamento e comunicou o cliente',
       'ótima, amei mais uma vez americanas arrasando na entrega. produto de otima qualidade recomendo muito.',
       'nao recebi o produto na data marcada. nao existi um telefone de facil acesso para se informar. e o atendimento ao cliente pelo e-mail é péssimo. nao tira suas dúvidas.',
       'até hoje não recebí o produto.... não recomendo a 

In [54]:
low_10 = np.argsort(clean_similarities)[:-10]